In [1]:
%load_ext sql
import sqlite3
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [2]:
%%sql
ATTACH DATABASE 'CourseData.db' AS coursedata;

 * sqlite:///CourseDataWarehouse.db
Done.


[]

# **Data Warehouse Design**

### Creating ***dimention*** tables

In [3]:
%%sql
-- Instructor table
DROP TABLE IF EXISTS INSTRUCTOR1;
CREATE TABLE INSTRUCTOR1 (
    Instructor_id INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Phone INTEGER,
    Email TEXT
);

-- Program table
DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    Program_id INTEGER PRIMARY KEY,
    Program_code TEXT NOT NULL,
    Program_name TEXT NOT NULL
);

-- Term table
DROP TABLE IF EXISTS TERM1;
CREATE TABLE TERM1 (
    Term_id INTEGER PRIMARY KEY,
    Term TEXT NOT NULL
);

-- Location table
DROP TABLE IF EXISTS LOCATION1;
CREATE TABLE LOCATION1 (
    Location_id INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [4]:
%%sql
DELETE FROM INSTRUCTOR1;
INSERT INTO INSTRUCTOR1(Instructor_id, Name, Phone, Email ) 
    SELECT Instructor_id, Name, Phone, Email 
    FROM coursedata.INSTRUCTOR;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1095 rows affected.


[]

%%sql
PRAGMA table_info();

In [6]:
%%sql
DELETE FROM PROGRAM1;
INSERT INTO PROGRAM1(Program_id, Program_code, Program_name) 
    SELECT Program_id, Program_code, Program_name 
    FROM coursedata.PROGRAM;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
83 rows affected.


[]

In [7]:
%%sql
DELETE FROM TERM1;
INSERT INTO TERM1(Term) 
    SELECT DISTINCT Term 
    FROM coursedata.CLASS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
19 rows affected.


[]

In [8]:
%%sql
DELETE FROM LOCATION1;
INSERT INTO LOCATION1(Location_id, Location) 
SELECT Location_id, Location
    FROM coursedata.LOCATION;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
207 rows affected.


[]

### Creating and modifying COURSE table 

In [9]:
%%sql
DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 AS SELECT * 
    FROM coursedata.COURSE 

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

%%sql
DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 AS SELECT * 
    FROM coursedata.COURSE 
    WHERE Catalog_year = 
        (SELECT DISTINCT(Catalog_year) 
             FROM coursedata.COURSE)

%%sql
CREATE TABLE COURSE1 AS SELECT * 
    FROM coursedata.COURSE

In [10]:
%%sql
PRAGMA table_info(COURSE1);

 * sqlite:///CourseDataWarehouse.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Course_id,INT,0,None,0
1,Catalog_year,TEXT,0,None,0
2,Program_id,INT,0,None,0
3,CatalogID,TEXT,0,None,0
4,Course_title,TEXT,0,None,0
5,Description,TEXT,0,None,0
6,Credits,REAL,0,None,0
7,Attributes,TEXT,0,None,0
8,Prerequisites,TEXT,0,None,0
9,Corequisites,TEXT,0,None,0


In [11]:
%%sql
PRAGMA foreign_keys=off;

--BEGIN TRANSACTION;

ALTER TABLE COURSE1 RENAME TO COURSE1_OLD;

CREATE TABLE COURSE1 (
    Course_id INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    Course_title TEXT NOT NULL, 
    Description TEXT,
    Program_id INTEGER,
    Credits REAL,
    Attributes TEXT,
    Prerequisites TEXT,
    Corequisites TEXT);

INSERT INTO COURSE1(Course_id, CatalogID, Course_title, Description, Program_id, Credits, Attributes, Prerequisites, Corequisites)
  SELECT Course_id, CatalogID, Course_title, Description, Program_id, Credits, Attributes, Prerequisites, Corequisites
  FROM COURSE1_OLD;

--COMMIT;

PRAGMA foreign_keys=on;

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
4440 rows affected.
Done.


[]

In [12]:
%%sql
PRAGMA table_info(COURSE1);

 * sqlite:///CourseDataWarehouse.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Course_id,INTEGER,0,None,1
1,CatalogID,TEXT,1,None,0
2,Course_title,TEXT,1,None,0
3,Description,TEXT,0,None,0
4,Program_id,INTEGER,0,None,0
5,Credits,REAL,0,None,0
6,Attributes,TEXT,0,None,0
7,Prerequisites,TEXT,0,None,0
8,Corequisites,TEXT,0,None,0


In [13]:
%%sql
DROP TABLE COURSE1_OLD

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [111]:
%%sql
SELECT * 
FROM COURSE1
WHERE (Course_id = 1766)

 * sqlite:///CourseDataWarehouse.db
Done.


Course_id,CatalogID,Course_title,Description,Program_id,Credits,Attributes,Prerequisites,Corequisites
1766,BB 0139,CFP Capstone Course,This course will engage the student in critical thinking and decision-making about personal financial management topics in the context of the financial planning process. You will gain the hands-on experience of taking a client from start to finish with their personal financial plan.,8,0 Credits,None,"BB 0130, BB 0132, BB 0134, BB 0136, BB 0138, BB 0140.",None


## Creating ***facts*** table CLASS_STATISTICS

In [103]:
%%sql
-- Class table
DROP TABLE IF EXISTS CLASS_STATISTICS;
CREATE TABLE CLASS_STATISTICS (
    ClassStat_id INTEGER PRIMARY KEY,
    Term_id INTEGER,
    Instructor_id INTEGER,
    Program_id INTEGER,
    Course_id INTEGER,
    Location_id INTEGER,
    crn INTEGER,
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    Meeting_times INTEGER,
    Meeting_hours INTEGER,
    --FOREIGN KEY (Term_id) REFERENCES TERM1(Term_id)
    FOREIGN KEY (Instructor_id) REFERENCES INSTRUCTOR1(Instructor_id)
    FOREIGN KEY (Program_id) REFERENCES PROGRAM1(Program_id)
    FOREIGN KEY (Location_id) REFERENCES LOCATION1(Location_id)
    FOREIGN KEY (Course_id) REFERENCES COURSE1(Course_id)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [105]:
%%sql
DELETE FROM CLASS_STATISTICS;
INSERT INTO CLASS_STATISTICS (Term_id, Instructor_id, Program_id, Course_id, Location_id, crn, Cap, Act, Rem, Meeting_times, Meeting_hours)
    SELECT DISTINCT TERM1.Term_id, coursedata.CLASS.Instructor_id, coursedata.COURSE.Program_id, coursedata.COURSE.Course_id,
            coursedata.MEETING.Location_id,  crn, Cap, Act, Rem,
        
            COUNT(coursedata.MEETING.Class_id) AS Meeting_times, 
           sum(strftime('%H:%M',coursedata.MEETING.End) - strftime('%H:%M',coursedata.MEETING.Start))AS Meeting_hours
        
    FROM coursedata.CLASS
        LEFT JOIN coursedata.COURSE ON (coursedata.COURSE.CatalogID = coursedata.CLASS.CatalogID)
        LEFT JOIN coursedata.MEETING ON (coursedata.CLASS.Class_id = coursedata.MEETING.Class_id)
        LEFT JOIN TERM1 USING(Term)
        WHERE coursedata.COURSE.Catalog_year <> '2018_2019'
    GROUP BY coursedata.MEETING.Class_id
 
        
        
       
        
        

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
12773 rows affected.


[]

%%sql
ALTER TABLE CLASS_STATISTICS
ADD COLUMN Term_id;

%%sql
INSERT INTO CLASS_STATISTICS (Term_id)
SELECT Term_id
    FROM TERM1 
    LEFT JOIN TERM1 ON (CLASS_STATISTICS.Term = TERM1.Term)

In [112]:
%%sql
SELECT *
FROM CLASS_STATISTICS
LIMIT 10



 * sqlite:///CourseDataWarehouse.db
Done.


ClassStat_id,Term_id,Instructor_id,Program_id,Course_id,Location_id,crn,Cap,Act,Rem,Meeting_times,Meeting_hours
1,1,None,4,1,None,71858,0,0,0,0,None
2,1,744,1,113,99,70384,0,31,-31,26,26
3,1,744,1,113,99,70385,0,31,-31,26,26
4,1,744,1,113,99,70382,0,31,-31,26,26
5,1,871,1,113,107,70291,0,29,-29,26,26
6,1,871,1,113,107,70350,0,30,-30,26,26
7,1,871,1,113,105,70381,0,31,-31,26,26
8,1,829,1,113,107,70383,30,31,-1,26,26
9,1,447,1,113,105,70391,30,32,-2,26,26
10,1,447,1,113,105,71105,30,33,-3,26,26


In [ ]:
%%sql 
SELECT *
    FROM CLASS
    WHERE (crn = 72340)


In [ ]:
%%sql
DELETE FROM CLASS_STATISTICS;
INSERT INTO CLASS_STATISTICS (Term, Instructor_id, Program_id, Course_id, Location_id, crn, Cap, Act, Rem, Meeting_times, Meeting_hours)
    SELECT DISTINCT coursedata.CLASS.Term, coursedata.CLASS.Instructor_id, coursedata.COURSE.Program_id, coursedata.COURSE.Course_id,
            coursedata.MEETING.Location_id,  crn, Cap, Act, Rem,
        
            COUNT(coursedata.MEETING.Class_id) AS Meeting_times, 
           sum(strftime('%H:%M',coursedata.MEETING.End) - strftime('%H:%M',coursedata.MEETING.Start))AS Meeting_hours
        
    FROM coursedata.CLASS
        LEFT JOIN coursedata.COURSE ON (coursedata.COURSE.CatalogID = coursedata.CLASS.CatalogID)
        LEFT JOIN coursedata.MEETING ON (coursedata.CLASS.Class_id = coursedata.MEETING.Class_id)
        WHERE coursedata.COURSE.Catalog_year <> '2018_2019'
    GROUP BY coursedata.MEETING.Class_id
 
        
        
       
        

%%sql
--DELETE FROM CLASS_STATISTICS;
--INSERT INTO CLASS_STATISTICS (Term_id, Instructor_id, Program_id, Location_id, Course_id, crn, Cap, Act, Rem)
    SELECT DISTINCT coursedata.CLASS.Term, coursedata.CLASS.Instructor_id, coursedata.COURSE.Program_id, coursedata.COURSE.Course_id, coursedata.MEETING.Location_id,  crn, Cap, Act, Rem, COUNT(coursedata.MEETING.Class_id) AS Meeting_times 
    FROM coursedata.CLASS
        LEFT JOIN coursedata.COURSE ON (coursedata.CLASS.Course_id = COURSE.Course_id)
        LEFT JOIN coursedata.MEETING ON (CLASS.Class_id = MEETING.Class_id)
        GROUP BY coursedata.MEETING.Class_id
            LIMIT 500
        
       
        

%%sql
--DELETE FROM CLASS_STATISTICS;
--INSERT INTO CLASS_STATISTICS (Term, Instructor_id, Program_id, Course_id, Location_id, crn, Cap, Act, Rem, Meeting_times)
    SELECT DISTINCT coursedata.CLASS.Term, coursedata.CLASS.Instructor_id, coursedata.COURSE.Program_id, coursedata.COURSE.Course_id, 
            coursedata.MEETING.Location_id,  crn, Cap, Act, Rem,
            COUNT(coursedata.MEETING.Class_id) AS Meeting_times 
          --SUM(strftime('%H:%M',coursedata.MEETING.End) - strftime('%H:%M',coursedata.MEETING.Start)) AS Hours
    FROM coursedata.CLASS
         JOIN coursedata.COURSE ON (coursedata.CLASS.Course_id = coursedata.COURSE.Course_id)
        LEFT JOIN coursedata.MEETING ON (coursedata.CLASS.Class_id = coursedata.MEETING.Class_id)
     GROUP BY coursedata.MEETING.Class_id
 
        
    LIMIT 100
    
        
       
        